In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
import sys as sys
import time
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import BasicTools as BT
import WaveTools as WT
import PlotTools as PT
import FFTTools as FFTT

In [2]:
nh = 2
h = 1. / nh
xCellFiller, wavesFiller = WT.MakeWaves(nh, h)
xCell = xCellFiller[0:nh - 1]
waves = wavesFiller[0:nh - 1, :]
nh = 2 * nh
print(xCell)
print(waves)
# xNode, wavesNode = WT.MakeNodeWaves(nh, h)
xBound = [[0, 2], [2, 4], [4, 6], [8, 12]] # [[0, 1], [0, 1, 2], [0, 1], [0, 1]] 
print(len(xBound[0]))# == [2, 2, 2, 2])

[0.25]
[[1.         0.63661977]]
2


In [38]:
def MakeAMRWaves(nh_max, xBound):
    x, y = BT.MakeXY(nh_max, xBounds = xBound)
    n = len(x) - 1
    xCell = 0.5 * (x[0:n] + x[1:n + 1])
    hDiag = x[1:n + 1] - x[0:n]
    waves = WT.CellWaves(nh, x)
    hs = np.zeros((n, n), float)
    np.fill_diagonal(hs, hDiag)
    zeroMat = np.zeros((n, nh_max - n), float)
    hMat = np.append(hs, zeroMat).reshape(nh_max, n)
    hMat[0, 0] = 1.
    waves = hMat @ waves
    return xCell, waves

In [41]:
xCell, waves = MakeAMRWaves(16, xBound)
PT.PlotWaves(nh, h, waves, xNode, waveXBasisNode, xBound = xBound)

NameError: name 'xNode' is not defined

In [24]:
blah = np.eye(5)
bluh = np.zeros((5, 3), float)
print(bluh)
print(np.append(blah, bluh).reshape(8, 5).T)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]


In [ ]:
print(np.any([i >= 16 for i in xBound]))